In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multitest import multipletests
import statsmodels.stats.multicomp as multicomp

In [2]:
data_df = pd.read_excel('../data/42255_2025_1311_MOESM5_ESM.xlsx', sheet_name='2L')

In [3]:
data_df

,HFD,HFD+SAL,HFD+SANA
0,5.130856,3.663540,0.054755
1,3.121056,3.612829,2.064112
2,1.605905,5.785027,1.014737
3,9.369681,4.725421,0.749991
4,8.379618,4.016210,1.246309
5,8.759274,3.082864,0.664874
6,3.884157,2.219070,1.049162
7,5.344271,3.075253,1.246309
8,6.802572,NaN,0.694845


In [4]:
grp1, grp2, grp3 = data_df['HFD'].dropna().values, \
data_df['HFD+SAL '].dropna().values, \
data_df['HFD+SANA '].dropna().values

In [6]:
collect = [grp1, grp2, grp3]

In [7]:
def reorder_label(label):
    if label[-2:] == 'ND':
        return 'ND vs ' + label.split('vs')[0].strip()
    else:
        return label

In [8]:
for cat in [0]:
    
    print('N samples: ' + str(len(np.concatenate(collect))))

    print('ANOVA:')
    f = scipy.stats.f_oneway(grp1, grp2, grp3)
    print('F(2, ' + str(len(np.concatenate(collect))-3) + ') = ' + str(f.statistic) + ', p = ' + str(f.pvalue))

    #print('Bonferroni:')
    comp = multicomp.MultiComparison(data=np.concatenate(collect), 
                                      groups=np.concatenate([['HFD']*len(grp1), 
                                                             ['HFD+SAL']*len(grp2), 
                                                             ['HFD+SANA']*len(grp3)]))
    tbl, a1, a2 = comp.allpairtest(scipy.stats.ttest_ind, method= "bonf", alpha=0.05)
    bonf = pd.DataFrame(tbl)
    bonf.columns = bonf.loc[0].astype(str)
    bonf = bonf[1:]
    bonf['pval_corr'] = a1[2]
    bonf['g1'] = bonf['group1']
    bonf['g2'] = bonf['group2']
    bonf_df = bonf.copy()
    bonf_df['Bonferroni p-value'] = bonf_df['pval_corr']
    bonf_df['Comparison'] = bonf_df['g1'].astype(str) + ' vs ' + bonf_df['g2'].astype(str)
    bonf_df['Comparison'] = bonf_df['Comparison'].apply(reorder_label)

    #print('Tukey:')
    tukey = scipy.stats.tukey_hsd(*collect).pvalue
    tukey_df = pd.DataFrame({'g1':['HFD', 'HFD', 'HFD+SAL'], 
     'g2':['HFD+SAL', 'HFD+SANA', 'HFD+SANA'],
     'Tukey p-value':[tukey[0][1], tukey[0][2], tukey[1][2]]})
    tukey_df['Comparison'] = tukey_df['g1'].astype(str) + ' vs ' + tukey_df['g2'].astype(str)

    combo_df = pd.merge(bonf_df, tukey_df, on=['Comparison'], how='outer')
    combo_df['Published p-value'] = ''
    combo_df = combo_df[['Comparison', 'Published p-value', 'Bonferroni p-value', 'Tukey p-value']].set_index('Comparison')
    print(combo_df.to_markdown())
    
    for grp in collect:
        print(np.mean(grp))

N samples: 26
ANOVA:
F(2, 23) = 17.71686243123135, p = 2.2041393293088297e-05
| Comparison          | Published p-value   |   Bonferroni p-value |   Tukey p-value |
|:--------------------|:--------------------|---------------------:|----------------:|
| HFD vs HFD+SAL      |                     |          0.191682    |     0.0579215   |
| HFD vs HFD+SANA     |                     |          0.000224525 |     1.38786e-05 |
| HFD+SAL vs HFD+SANA |                     |          1.89518e-05 |     0.00806203  |
5.821932222222221
3.77252675
0.9761214822222222


Here is Figure 2L:

![file](../img/pub_fig_2l.png)

This is described as involving a one-way ANOVA followed by Bonferroni post-hoc. Using the [source data](https://www.nature.com/articles/s42255-025-01311-z#Sec38), we arrive at F(2, 23) = 17.7, p = 2.20E-5. However, the p-values shown for the post-hoc test are incorrect for Bonferroni post-hoc comparison (i.e., independent t-tests between groups and Bonferroni FWER correction on p-values for each pairwise test) and the p-value shown for HFD vs HFD+SANA is incorrect for for Tukey's post-hoc test. See summary table below.

| Comparison          | Published p-value   |   Bonferroni p-value |   Tukey p-value |
|:--------------------|:--------------------|---------------------:|----------------:|
| HFD vs HFD+SAL      |                     |          0.192    |     0.0579215   |
| HFD vs HFD+SANA     |                     |          0.000225 |     1.39-5 |
| HFD+SAL vs HFD+SANA |                     |          1.90E-5 |     0.00806  |

Could the authors clarify? 

The code for this analysis is available at [github.com/reeserich/cal_et_al_2025](https://github.com/reeserich/cal_et_al_2025).